## Preview Streaming data from MessageHub

In [ ]:
from pixiedust.apps.messageHub import *

MessageHubStreamingApp().run(credentials={
    "username": "XXX",
    "password": "XXX",
    "api_key" : "XXX"
})